In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../scripts'))
from helpers import build_audio_features_data
sys.path.insert(0,'..')
import pandas as pd
from itertools import product


PLAN:

I am going to extract audio features for each genre, using the genres by artists. 

I will look at the distributions and calculate an average to see whether I can use the mean value for clustering. I will then perform some sort of clustering algorithim on these mean values for audio features to see if we can calculate some sort of similarity score for genres. This will be interesting on its own for some sort of visulisations, but more importantly we can use this for our custom loss function which will take into account the similarity between genres so as not to unfairly evaluate the models performance and hinder learning.

## Step 1 - Extract Audio Features by Genre


In [2]:

PLAYLIST_LIST = ["4AoI1VZwQsSKFXSQnAgxtV","3ZWRMtzrap0EosVqsU6IW3","4rnleEAOdmFAbRcNCgZMpY"]


In [3]:
df = build_audio_features_data(PLAYLIST_LIST)


                          id  acousticness  \
0     2Oy1pcU49bb706RUWLxQpK      0.039500   
1     3ujsqiZbDDs9w7W0KGKhvm      0.372000   
2     7lgJwN0BuZFtKv3v1jH3eR      0.937000   
3     1fNEWpJKK8bdACgY4B1SFY      0.542000   
4     12PAgtalaIQid82B37CXjT      0.352000   
...                      ...           ...   
5195  4uoumbAMEMaKdtiv763jKz      0.023500   
5196  1louJpMmzEicAn7lzDalPW      0.057500   
5197  4I5JjW4KpDiC7kJBpCmxhH      0.024600   
5198  5xbFkdL9vW2uPlBXL3Upw0      0.079000   
5199  0C5ccaa4Jh8mAk9lcPXm1G      0.000498   

                                           analysis_url  danceability  \
0     https://api.spotify.com/v1/audio-analysis/2Oy1...         0.219   
1     https://api.spotify.com/v1/audio-analysis/3ujs...         0.245   
2     https://api.spotify.com/v1/audio-analysis/7lgJ...         0.186   
3     https://api.spotify.com/v1/audio-analysis/1fNE...         0.245   
4     https://api.spotify.com/v1/audio-analysis/12PA...         0.305   
...      

## Step 2 - Statistical Analysis

In [9]:
def aggregate_genres(track_audio_data):
    
    data = pd.DataFrame(
        columns=[
            col
            for col in track_audio_data.columns
            if not track_audio_data[col].isnull().any()
        ]
    )
    for col in track_audio_data.columns:
        if track_audio_data[col].isnull().any():
            data.loc[col] = track_audio_data[track_audio_data[col].notnull()].aggregate(
                func="mean"
            )
    return data


def get_genre_statistics(track_audio_data):
    
    genre_stats = pd.DataFrame()
    
    genre_cols =[
            col
            for col in track_audio_data.columns
            if track_audio_data[col].isnull().any()
        ]

    audio_cols = [col for col in track_audio_data.columns if col not in genre_cols ]
    audio_cols = [col for col in audio_cols if col not in ["mode","tempo","time_signature","duration_ms","key"]]
    
    

    for genre in genre_cols:
        genre_df = track_audio_data[track_audio_data[genre].notnull()][audio_cols]

        genre_stats.loc[genre, "num samples"] = genre_df.shape[0]
        for audio_feature in audio_cols:

            
            genre_stats.loc[genre,f"{audio_feature} std"] = genre_df[audio_feature].std()
            # genre_stats.loc[genre,f"{audio_feature} var"] = genre_df[audio_feature].var()


    genre_stats.drop(
        genre_stats[genre_stats["num samples"] < 10].index,inplace=True
    )
    
    return genre_stats.sort_values("num samples")


stats = get_genre_statistics(df)
stats




,num samples,acousticness std,danceability std,energy std,instrumentalness std,liveness std,loudness std,speechiness std,valence std
alternative metal,10.0,0.003668,0.040701,0.041812,4.300097e-02,0.101626,0.917200,0.068362,0.174095
pop rock,10.0,0.256840,0.111812,0.209742,1.668633e-05,0.075061,1.385556,0.008264,0.222249
classical performance,10.0,0.131793,0.098497,0.129049,3.297198e-01,0.237338,3.231957,0.006644,0.125724
finnish hip hop,10.0,0.063096,0.095925,0.119259,4.964776e-07,0.076512,1.538527,0.087336,0.193116
finnish pop,10.0,0.063096,0.095925,0.119259,4.964776e-07,0.076512,1.538527,0.087336,0.193116
progressive electro house,10.0,0.040503,0.148321,0.101267,3.166929e-01,0.240420,1.638540,0.111544,0.200371
finnish dance pop,10.0,0.063096,0.095925,0.119259,4.964776e-07,0.076512,1.538527,0.087336,0.193116
electropop,11.0,0.212475,0.058792,0.148076,4.220380e-02,0.172374,0.935635,0.031226,0.162593
modern rock,11.0,0.242943,0.135210,0.231745,1.979505e-04,0.176015,2.159355,0.017809,0.277190
dark trap,11.0,0.262648,0.116496,0.100096,1.207455e-01,0.097743,2.042344,0.193125,0.214629


Some features have a larger spread than others, will experiment with just using a subset of these for classification and clustering.

In [10]:
print("Mean stds:\n",stats.mean(0).sort_values())
print("Median Stds\n"),stats.median(0).sort_values()


Mean stds:
 speechiness std          0.075167
danceability std         0.112832
liveness std             0.134335
instrumentalness std     0.136249
energy std               0.141096
acousticness std         0.183349
valence std              0.186429
loudness std             2.611727
num samples             23.400000
dtype: float64
Median Stds



(None,
 speechiness std          0.086117
 danceability std         0.104432
 instrumentalness std     0.117389
 energy std               0.128407
 liveness std             0.131461
 acousticness std         0.189568
 valence std              0.193116
 loudness std             1.820560
 num samples             19.000000
 dtype: float64)

We can see which features may be a better choice than others in terms of data spread, although they may not be a good predictor of category.

In [25]:
features = ["speechiness","danceability","liveness","instrumentalness","energy"]
genre_list = stats[stats["num samples"] > 5].index
genre_list

Index(['alternative metal', 'pop rock', 'classical performance',
       'finnish hip hop', 'finnish pop', 'progressive electro house',
       'finnish dance pop', 'electropop', 'modern rock', 'dark trap',
       'electro house', 'pops orchestra', 'easy listening', 'tropical house',
       'sad rap', 'edm', 'pop dance', 'scorecore', 'trap', 'electro',
       'filter house', 'rap', 'melodic rap', 'post-teen pop', 'miami hip hop',
       'pop rap', 'emo rap', 'dance pop', 'pop', 'soundtrack'],
      dtype='object')

## Step 3 - Fit Data to Clustering Model



In [26]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler, normalize
from collections import defaultdict

In [27]:
agg = aggregate_genres(df)
temp = agg.loc[genre_list,features]
temp

,speechiness,danceability,liveness,instrumentalness,energy
alternative metal,0.084550,0.509900,0.170060,1.361713e-02,0.913600
pop rock,0.041130,0.558600,0.151440,7.589000e-06,0.686600
classical performance,0.042740,0.265900,0.233560,6.612420e-01,0.191310
finnish hip hop,0.093050,0.650900,0.176930,1.570000e-07,0.776400
finnish pop,0.093050,0.650900,0.176930,1.570000e-07,0.776400
progressive electro house,0.104880,0.585200,0.342200,2.066165e-01,0.842200
finnish dance pop,0.093050,0.650900,0.176930,1.570000e-07,0.776400
electropop,0.055864,0.690364,0.200255,1.275086e-02,0.762818
modern rock,0.047136,0.575909,0.186700,9.067455e-05,0.692455
dark trap,0.211582,0.651545,0.133900,3.697119e-02,0.566000


In [28]:



temp_df = pd.DataFrame(normalize(temp), columns = temp.columns)
temp_df = StandardScaler().fit_transform(temp_df)


model = DBSCAN(min_samples=2,eps=0.7).fit(temp_df)
labels = model.labels_
labels

label_dict = dict(zip(temp.index,labels))


res = defaultdict(list)
for key, val in sorted(label_dict.items()):
    res[val].append(key)

res

defaultdict(list,
            {-1: ['alternative metal',
              'classical performance',
              'dark trap',
              'melodic rap',
              'sad rap',
              'scorecore'],
             0: ['dance pop', 'electropop', 'modern rock', 'pop', 'pop rock'],
             3: ['easy listening', 'pops orchestra', 'soundtrack'],
             2: ['edm',
              'electro house',
              'pop dance',
              'progressive electro house',
              'tropical house'],
             5: ['electro', 'filter house'],
             6: ['emo rap', 'miami hip hop'],
             1: ['finnish dance pop',
              'finnish hip hop',
              'finnish pop',
              'pop rap',
              'post-teen pop'],
             4: ['rap', 'trap']})